In [1]:
import pandas as pd
import numpy as np
import igraph as ig
import copy
import random

In [2]:
df_clientes = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="clientes")
df_pedidos = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [7]:
class Pedido(object):
    
    def __init__(self, ix, x, y, carga):
        self.ix = ix
        self.x = x
        self.y = y
        self.carga = carga
        self.asignado = False
        self.camion_ix = None
        
    def __str__(self):
        return f'Pedido {self.ix}\nCarga {self.carga} tn\nAsignado {self.asignado}\nAsignado a Camion {self.camion_ix}'
    
    def __repr__(self):
        return f'Pedido {self.ix}\nCarga {self.carga} tn\nAsignado {self.asignado}\nAsignado a Camion {self.camion_ix}'
    
    def get_ix(self):
        return self.ix
    
    def get_carga(self):
        return self.carga
    
    def distancia(self, other):
        dist = ((self.x - other.x)**2 + (self.y - other.y)**2)**(1/2)
        return round(dist, 1)
    
def distancia(ped1, ped2):
    dist = ((ped1.x - ped2.x)**2 + (ped1.y - ped2.y)**2)**(1/2)
    return round(dist, 1)

In [8]:
df_pedido1 = df_pedidos[["cliente", "pedidos1", "coord_x", "coord_y"]]
df_pedido1.head()

,cliente,pedidos1,coord_x,coord_y
0,A,4,0.257571,1.803726
1,B,6,1.523313,2.102301
2,C,1,0.710910,2.629800
3,D,8,1.013940,2.149740
4,E,4,0.609480,0.666810


In [ ]:
pedidos_tomados = "pedidos1"
lista_pedidos = [Pedido(ix=row.cliente, x=row.coord_x, y=row.coord_y, carga=row[pedidos_tomados]) for _, row in df_pedidos.iterrows() if row[pedidos_tomados] != 0]
lista_pedidos

In [ ]:
# Las aristas serán las uniones entre pedidos que pueden estar juntos.
edges = [()]

In [5]:
locations = [(x, y) for x, y in zip(df_pedido1.coord_x, df_pedido1.coord_y)]
print(locations)
g1 = ig.Graph(locations)